In [2]:
# Already tested and implemented the scrapping process on jupyter notebook, copy paste it here. 67k entries were scrapped
# I still need to examine the data to make sure it is up to date and no discrepancies
# Display names needs to be added, or i think it was extensionName

# extensions after the 8th are not being included. Another I could do it -> infinite scroll and gather each title name, then append it to the url and get the info. I still do not know why the below doesnt work rn
import pandas as pd
import requests
from datetime import datetime
from dateutil import parser


vscodeDataSet = pd.DataFrame(columns = ['Extension Name', 'Publisher Name', 'DisplayPublisherName','DisplayExtensionName','Downloads','Rating','Verified','Published','Last Update']) # I might need more cols later
url = "https://marketplace.visualstudio.com/_apis/public/gallery/extensionquery" # scrapping from this url
headers = {
        "Accept": "application/json;api-version=3.0-preview.1",
        "Content-Type": "application/json"
    }
# 88 was found through trial and error. This is the maximum number of pages that can be scrapped 
for i in range(0,89):
    payload = {
            "filters": [
                {
                    "criteria": [
                        {"filterType": 8, "value": "Microsoft.VisualStudio.Code"}],
                    'pageNumber':i,
                    'pageSize':1000,
                    
                }
            ],
            "flags": 870
        }
    make_request = requests.post(url,headers=headers,json=payload)
    database = make_request.json()

    for x in range(0,1000):
        # By looking at the json file, this is how I can access each field that I want.
            extensionName = database['results'][0]['extensions'][x]['extensionName']
            publisherName = database['results'][0]['extensions'][x]['publisher']['publisherName']
            displayedExtensionName = database['results'][0]['extensions'][x]['displayName']
            displayedPublisherName = database['results'][0]['extensions'][x]['publisher']['displayName']
            try:
                downloads = database['results'][0]['extensions'][x]['statistics'][0]['value']
                rating = database['results'][0]['extensions'][x]['statistics'][-2]['value']
            except Exception as e:
                downloads = 0
                rating = 0
            
            verified = database['results'][0]['extensions'][x]['publisher']['isDomainVerified']
            published = database['results'][0]['extensions'][x]['releaseDate']
            lastUpdate = database['results'][0]['extensions'][x]['lastUpdated']
            date = parser.isoparse(published)
            timestamp = int(datetime.timestamp(date))
            new_row = pd.DataFrame([[extensionName,publisherName,displayedPublisherName,displayedExtensionName,downloads,rating,verified,timestamp,lastUpdate]], columns=['Extension Name', 'Publisher Name', 'DisplayPublisherName','DisplayExtensionName','Downloads','Rating','Verified','Published','Last Update'])
            vscodeDataSet = pd.concat([vscodeDataSet,new_row])

C:\Users\Ziyad\AppData\Local\Temp\ipykernel_11996\2972472533.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vscodeDataSet = pd.concat([vscodeDataSet,new_row])


In [14]:
vscodeDataSet['Published'] = vscodeDataSet['Published'].sort_values(ascending=True)
vscodeDataSet.drop_duplicates(keep='first',inplace=True) # 1-2k duplicates found
vscodeDataSet.isnull().sum() # no values that are null
# Filter the DataFrame to include only rows where 'Downloads' is less than 1,000,000
filtered_vscodeDataSet = vscodeDataSet[vscodeDataSet['Downloads'] < 1000000]

# If you want to count the number of such rows, you can use the len() function
count = len(filtered_vscodeDataSet)
print(count)


88107


In [ ]:
"""url = 'https://marketplace.visualstudio.com/items?itemName='
for i,j in vscodeDataSet.iterrows():
    url = f'https://marketplace.visualstudio.com/items?itemName={j['Publisher Name']}.{j['Extension Name']}'
    vscode_request = requests.get(url=url)
    print(vscode_request.status_code)"""